In [ ]:
%load_ext lab_black
import mne
import numpy as np
import plotly.graph_objects as go

# import tensorflow as tf
from ipywidgets import Dropdown, FloatRangeSlider, IntSlider, interact
from mne.datasets import sample
from scipy import signal
from scipy.interpolate import interp1d

np.set_printoptions(precision=2, linewidth=150)
base_layout = dict(
    template="plotly_dark",
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    margin=dict(l=20, r=20, t=20, b=20),
    dragmode="select",
    showlegend=False,
    width=900,
    height=400,
)

In [ ]:
data_path = sample.data_path()
raw_fname = data_path + "/MEG/sample/sample_audvis_filt-0-40_raw.fif"
event_fname = data_path + "/MEG/sample/sample_audvis_filt-0-40_raw-eve.fif"
raw = mne.io.read_raw_fif(raw_fname)

raw.pick_types(meg=False, eeg=True, eog=True).load_data()

# # This particular dataset already has an average reference projection added
# # that we now want to remove it for the sake of this example.
# raw.set_eeg_reference([])

reject = dict(eeg=180e-6, eog=150e-6)
event_dict = {
    "auditory/left": 1,
    "auditory/right": 2,
    "visual/left": 3,
    "visual/right": 4,
}
tmin, tmax = -0.2, 0.8
events = mne.read_events(event_fname)
all_epochs = mne.Epochs(
    raw, events=events, event_id=event_dict, tmin=tmin, tmax=tmax, reject=reject
)
all_epochs.load_data()

sampling_freq = 1 / (all_epochs.times[1] - all_epochs.times[0])

In [ ]:
channel_locations = np.array([ch["loc"][:3] for ch in raw.info["chs"]])
x, y, z = channel_locations.T
scalp3d = go.FigureWidget(layout=base_layout)
scalp3d.update_layout(width=300, height=300)
scalp3d.add_scatter3d(
    x=x,
    y=y,
    z=z,
    mode="markers+text",
    text=list(range(len(x))),
    marker_size=4,
    #     hoverinfo="skip",
)
# scalp3d.add_scatter3d(x=x, y=y, z=z, mode='markers')

In [ ]:
scalp = go.FigureWidget(layout=base_layout)
scalp.update_layout(width=300, height=300)
scalp.add_scatter(x=x, y=y, mode="text", hoverinfo="skip", text=list(range(len(x))))
scalp.data[0].selectedpoints = (0,)
scalp
# select some points by dragging on the plot

In [ ]:
def dist(f1, f2):
    assert f1.shape == f2.shape
    diff = f1 - f2
    return np.dot(diff, diff) ** (1 / 2) * 1e6
    # return np.dot(diff, diff) * 1e12


def mask(array, window):
    mapping = interp1d([tmin, tmax], [0, len(array)])
    min_index, max_index = mapping(window)
    return np.array(
        [el if min_index < i < max_index else 0 for i, el in enumerate(array)]
    )


def band_pass(array, freq_range):
    sos = signal.butter(6, freq_range, "bandpass", fs=sampling_freq, output="sos")
    return signal.sosfiltfilt(sos, array)


def extract_erp(epoch, selected_chs, band_pass_range, window):
    filtered = epoch[selected_chs].mean(axis=0)
    filtered = band_pass(filtered, band_pass_range)
    filtered = mask(filtered, window)
    return filtered

In [ ]:
channel_num = 26
max_amp = 0.000008

# weights = np.ones((1, 60)) / 60
# dist(clean, weights @ epoch0)

conditions = list(all_epochs.event_id.keys())
max_cond_count = max(all_epochs[cond].events.__len__() for cond in conditions)

fig = go.FigureWidget(layout=base_layout)
fig.update_layout(
    xaxis_range=[tmin, tmax],
    yaxis_range=[-max_amp, max_amp],
)
fig.add_scatter(
    x=all_epochs.times,
    hoverinfo="skip",
    mode="lines+markers",
    marker_opacity=0,
    line_color="red",
    line_width=4,
)
for _ in range(max_cond_count):
    fig.add_scatter(x=all_epochs.times, hoverinfo="skip", line_width=1, opacity=0)
window = FloatRangeSlider(value=[tmin, tmax], min=tmin, max=tmax)

# setting window by dragging:
# def set_range(trace, points, selector):
#     window.value = selector.xrange
# fig.data[0].on_selection(set_range)

cond_selector = Dropdown(options=conditions)


@interact(condition=cond_selector)
def set_condition(condition):
    with fig.batch_update():
        epochs = all_epochs[condition]._data
        grand_average = epochs.mean(axis=0)[channel_num]
        for trace in fig.data[1:]:
            trace.opacity = 0
        fig.data[0].y = grand_average


# TODO max_cond_count should be changed accoring to the condition used
# ensure, set_condition is executed first, when changing condition
@interact(
    epoch_num=IntSlider(value=1, min=0, max=max_cond_count - 1),
    band_pass_range=FloatRangeSlider(value=[0.1, 20], min=0.1, max=50),
    window=window,
    condition=cond_selector,
    many_epochs=False,
)
def update_plots(epoch_num, band_pass_range, window, condition, many_epochs):
    with fig.batch_update():
        selected_chs = list(scalp.data[0].selectedpoints)
        epochs = all_epochs[condition]._data
        # dist_total = 0
        for i, epoch in enumerate(epochs):
            i += 1  # i=0 is grand_average
            if i <= epoch_num and many_epochs or i == epoch_num and not many_epochs:
                filtered = extract_erp(epoch, selected_chs, band_pass_range, window)
                fig.data[i].y = filtered
                fig.data[i].opacity = 1 / (epoch_num ** (1 / 3))
                # dist_total += dist(grand_average, filtered)
            else:
                fig.data[i].opacity = 0
        # print(f' distance: {dist_total:9.0f}')


fig

In [ ]:
# scalp.observe(update_plots)